In [1]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

In [2]:
def get_features_targets(data):
    features = np.zeros((data.shape[0],4))
    features[:,0] = data['u'] - data['g']
    features[:,1] = data['g'] - data['r']
    features[:,2] = data['r'] - data['i']
    features[:,3] = data['i'] - data['z']

    targets = data['redshift']

    return features, targets 

In [3]:
def median_diff(predicted, actual):
    return np.median(np.abs(predicted - actual))

In [4]:
def cross_validate_model(model, features, targets, k):
    kf = KFold(n_splits=k, shuffle=True)

    median_diffs = list()

    for train_indices, test_indices in kf.split(features):
        train_features, test_features = features[train_indices], features[test_indices]
        train_targets, test_targets = targets[train_indices], targets[test_indices]

        model.fit(train_features, train_targets)
        median_diffs.append(median_diff(model.predict(test_features), test_targets))

    return median_diffs

In [7]:
def split_galaxies_qsos(data):
    galaxies = data[data['spec_class'] == b'GALAXY']
    qso = data[data['spec_class'] == b'QSO']
    return galaxies, qso

In [8]:
def cross_validate_median_diff(data):
  features, targets = get_features_targets(data)
  dtr = DecisionTreeRegressor(max_depth=19)
  return np.mean(cross_validate_model(dtr, features, targets, 10))

In [9]:
data = np.load('./sdss_galaxy_colors.npy')

# Split the data set into galaxies and QSOs
galaxies, qsos= split_galaxies_qsos(data)

# Here we cross validate the model and get the cross-validated median difference
# The cross_validated_med_diff function is in "written_functions"
galaxy_med_diff = cross_validate_median_diff(galaxies)
qso_med_diff = cross_validate_median_diff(qsos)

# Print the results
print("Median difference for Galaxies: {:.3f}".format(galaxy_med_diff))
print("Median difference for QSOs: {:.3f}".format(qso_med_diff))

Median difference for Galaxies: 0.016
Median difference for QSOs: 0.075
